## Initialise Backends

In [ ]:
from pytket.extensions.myqos import Myqos, MyqosBackend
from pytket.extensions.myqos import QuantinuumConfig, AerConfig

n_shots = 10000

myqos = Myqos()

H1E_configuration = QuantinuumConfig(
    device_name="H1-1E",
    user_group='DEFAULT',
    noisy_simulation=True,
    simulator='state-vector',
)
aer_configuration = AerConfig(user_group='DEFAULT')

experiment = myqos.get_experiment_by_name("test_hidden_inverse")

h1e_backend = MyqosBackend(H1E_configuration, experiment=experiment, remote=True)
aer_backend = MyqosBackend(aer_configuration, experiment=experiment, remote=True)

## Construct CZ Gate and Inverse

In [ ]:
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter

CX = Circuit(2)
CX.PhasedX(3.5, 1.5, 1)
CX.ZZPhase(0.5, 0,1)
CX.Rz(0.5, 0)
CX.PhasedX(2.5, 0, 1)
CX.Rz(3.5, 1)
render_circuit_jupyter(CX)

hi_CX = CX.copy().dagger()
render_circuit_jupyter(hi_CX)

## Construct CZ Sequence Circuits

In [ ]:
orig_circ = Circuit(2)
hi_circ = Circuit(2)

# Apply Hadamard at the begining
orig_circ.PhasedX(0.5, -0.5, 0)
orig_circ.Rz(1, 0)
orig_circ.PhasedX(0.5, -0.5, 1)
orig_circ.Rz(1, 1)

# Apply Hadamard at the begining
hi_circ.PhasedX(0.5, -0.5, 0)
hi_circ.Rz(1, 0)
hi_circ.PhasedX(0.5, -0.5, 1)
hi_circ.Rz(1, 1)

n_pairs = 20
for _ in range(n_pairs):
    orig_circ.add_barrier([0,1])
    orig_circ.append(CX)
    orig_circ.add_barrier([0,1])
    orig_circ.append(CX)
    orig_circ.add_barrier([0,1])
    
    hi_circ.add_barrier([0,1])
    hi_circ.append(CX)
    hi_circ.add_barrier([0,1])
    hi_circ.append(hi_CX)
    hi_circ.add_barrier([0,1])

# Apply Hadamards at end and measure
orig_circ.PhasedX(0.5, -0.5, 0)
orig_circ.Rz(1, 0)
orig_circ.PhasedX(0.5, -0.5, 1)
orig_circ.Rz(1, 1)
orig_circ.measure_all()

# Apply Hadamards at end and measure
hi_circ.PhasedX(0.5, -0.5, 0)
hi_circ.Rz(1, 0)
hi_circ.PhasedX(0.5, -0.5, 1)
hi_circ.Rz(1, 1)
hi_circ.measure_all()

render_circuit_jupyter(orig_circ)
render_circuit_jupyter(hi_circ)

## Run and Compare Counts

In [ ]:
result_handle = h1e_backend.process_circuit(orig_circ, n_shots=n_shots)
hi_result = h1e_backend.get_result(result_handle)
hi_counts = hi_result.get_counts()
hi_counts

In [ ]:
result_handle = h1e_backend.process_circuit(hi_circ, n_shots=n_shots)
hi_result = h1e_backend.get_result(result_handle)
hi_counts = hi_result.get_counts()
hi_counts